In [4]:
import dataloader2
import numpy as np
import os
len(dataloader2.good_sids)

37

In [2]:
train_sids = np.random.choice(dataloader2.good_sids, 30)
test_sids = [i for i in dataloader2.good_sids if i not in train_sids]

In [3]:
from models import model1min

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1, 14, 32)         800       
                                                                 
 max_pooling2d (MaxPooling2  (None, 1, 4, 32)          0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 2)                 22        
                                                                 
Total params: 2112 (8.25 KB)
Trainable params: 2112 (8.25 KB)
Non-trainable params: 0 (0.00 Byte)
________________________

In [12]:
# if model1min.is_not_trained():
from tqdm.notebook import tqdm
training_files = [os.path.join("data", "sids", f"{sid}.parquet") for sid in tqdm(train_sids)]
training_XandIndex = [model1min.getXandIndex(sid_file) for sid_file in tqdm(training_files)]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [13]:
import pandas as pd

dfs = (dataloader2.load_sid(sid_f) for sid_f in tqdm(training_files))
training_y = [(df['activity']=='Sleeping').astype(int) for df in tqdm(dfs, total=len(training_files))]

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [16]:
training_y = [y.loc[index] for y,(_,index) in zip(training_y, training_XandIndex)]

In [17]:
len(training_XandIndex), len(training_y)

(30, 30)

In [20]:
all([training_XandIndex[i][0].shape[0]==training_y[i].shape[0] for i in range(30)])

True

In [24]:
X = np.concatenate([i[0] for i in training_XandIndex])
y = np.concatenate(training_y)
X.shape, y.shape

((11109780, 2, 25), (11109780,))

In [28]:
val_size = sum([len(i) for i in training_y[-2:]])
from sklearn.utils import shuffle
x_train, y_train = shuffle(X[:-val_size], y[:-val_size], random_state=42)
x_test, y_test = X[-val_size:], y[-val_size:]

for i in [x_train, y_train, x_test, y_test]:
    print(i.shape)

(10302708, 2, 25)
(10302708,)
(807072, 2, 25)
(807072,)


In [8]:
if model1min.is_not_trained():
    import keras
    model1min.model.compile(
        optimizer='adam',
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'],
        jit_compile=True
    )
    cp_callback = keras.callbacks.ModelCheckpoint(
        model1min.checkpoint_file,
        save_weights_only=True,
        verbose=1
    )
    hist = model1min.model.fit(
        x_train,
        y_train,
        epochs=10,
        validation_data=(x_test, y_test),
        callbacks=[cp_callback]
    )

False